In [ ]:
#import tensorflow as tf
import math

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print('TensorFlow Version:', tf.__version__) 

In [ ]:
excel_path = r"Data.xlsx"

FPX_Data = pd.read_excel(excel_path, sheet_name = None)

In [ ]:
FPX_Train = FPX_Data['Train'] 
colnames = np.array(FPX_Train.columns)
FPX_Train = np.array(FPX_Train)
FPX_Train_X = FPX_Train[:,2:]
FPX_Train_y = FPX_Train[:,1]
FPX_Train_crudes = FPX_Train[:,0]

FPX_Test = FPX_Data['Test']
FPX_Test = np.array(FPX_Test)
FPX_Test_X = FPX_Test[:,2:]
FPX_Test_y = FPX_Test[:,1]
FPX_Test_crudes = FPX_Test[:,0]

FPX_Val = FPX_Data['Validation']
FPX_Val = np.array(FPX_Val)
FPX_Val_X = FPX_Val[:,2:]
FPX_Val_y = FPX_Val[:,1]
FPX_Val_crudes = FPX_Val[:,0]

FPX_Rem = FPX_Data['Validation 2'] 
FPX_Rem = np.array(FPX_Rem)
FPX_Rem_X = FPX_Rem[:,2:]
FPX_Rem_y = FPX_Rem[:,1]
FPX_Rem_crudes = FPX_Rem[:,0]

In [ ]:
# Helper functions

def get_r2(x_list, y_list):
    n = len(x_list)
    x_bar = sum(x_list)/n
    y_bar = sum(y_list)/n
    x_std = math.sqrt(sum([(xi-x_bar)**2 for xi in x_list])/(n-1))
    y_std = math.sqrt(sum([(yi-y_bar)**2 for yi in y_list])/(n-1))
    zx = [(xi-x_bar)/x_std for xi in x_list]
    zy = [(yi-y_bar)/y_std for yi in y_list]
    r = sum(zxi*zyi for zxi, zyi in zip(zx, zy))/(n-1)
    return r**2

def plot_results(history):
    plt.figure(figsize=(12, 4))
    epochs = len(history['val_loss'])
    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), history['val_loss'], label='Val Loss')
    plt.plot(range(epochs), history['train_loss'], label='Train Loss')
    plt.xticks(list(range(epochs)))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(range(epochs), history['val_acc'], label='Val Acc')
    plt.xticks(list(range(epochs)))
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    return plt

In [ ]:
def loss_function(y_true, y_pred, name='squared_error', threshold=1):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    absolute_loss = tf.abs(error)
    linear_loss = threshold * absolute_loss - threshold**2 / 2
    
    if name == 'squared_error':
        return squared_loss
    elif name == 'absolute_error':
        return absolute_loss
    elif name == 'huber_loss':
        return tf.where(is_small_error, squared_loss, linear_loss)
    else:
        return None

def weight_initializer(shape, name='default_normal', minval_unif=0, maxval_unif=None, dtype=tf.float32):
    if name == 'default_normal':
        return tf.random.normal(shape, dtype=dtype)
    elif name == 'default_uniform':
        return tf.random.uniform(shape, minval=minval_unif, maxval=maxval_unif, dtype=dtype)
    elif name == 'glorot_normal':
        stddev = tf.sqrt(2. / (shape[0] + shape[1]))
        return tf.random.normal(shape, stddev=stddev, dtype=dtype)
    elif name == 'glorot_uniform':
        r = tf.sqrt(6. / (shape[0] + shape[1]))
        return tf.random.uniform(shape, minval=-r, maxval=r, dtype=dtype)
    elif name == 'he_normal':
        stddev = tf.sqrt(2. / (shape[0]))
        return tf.random.normal(shape, stddev=stddev, dtype=dtype)
    elif name == 'he_uniform':
        r = tf.sqrt(6. / (shape[0]))
        return tf.random.uniform(shape, minval=-r, maxval=r, dtype=dtype)
    elif name == 'lecun_normal':
        stddev = tf.sqrt(1. / (shape[0]))
        return tf.random.normal(shape, stddev=stddev, dtype=dtype)
    elif name == 'lecun_uniform':
        r = tf.sqrt(3. / (shape[0]))
        return tf.random.uniform(shape, minval=-r, maxval=r, dtype=dtype)
    else:
        return None

def logit(z):
    return 1 / (1 + np.exp(-z))

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

def relu(weights): 
    return np.where(weights < 0., np.zeros_like(weights), weights)

def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)

def selu(z, scale=1.0507, alpha=1.6732):
    return scale * elu(z, alpha)

def l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [ ]:
class NeuralNetwork:
    # Initialization of network parameters
    def __init__(self, layers):
        self.layers = layers
        self.L = len(layers)
        self.num_features = layers[0]
        
        self.W = {}
        self.b = {}
        
        self.dW = {}
        self.db = {}
        
        self.setup()
       
    # Initialize the weights 
    def setup(self):
        for i in range(1, self.L):
            self.W[i] = tf.Variable(weight_initializer(shape=(self.layers[i], self.layers[i-1])), name='lecun_uniform')
            self.b[i] = tf.Variable(weight_initializer(shape=(self.layers[i], 1), name='lecun_uniform'))
    
    # Forward Pass to iterate over each layer and get the final layer activation output
    def forward_pass(self, X):
        A = tf.convert_to_tensor(X, dtype=tf.float32)
        for i in range(1, self.L):
            Z = tf.matmul(A, tf.transpose(self.W[i])) + tf.transpose(self.b[i])
            if i != self.L-1:
                A = tf.nn.sigmoid(Z)
            else:
                A = Z
        return A
    
    # Loss function: 
    def compute_loss(self, A, Y):
        loss = loss_function(y_true=Y, y_pred=A, name='squared_error')
        return tf.reduce_mean(loss)
    
    # Weight & Bias update using learning rate and their respective gradients.
    def update_params(self, lr):
        for i in range(1, self.L):
            self.W[i].assign_sub(lr * self.dW[i])
            self.b[i].assign_sub(lr * self.db[i])
    
    # Predict function: Forward pass --> ReLu
    def predict(self, X):
        A = self.forward_pass(X)
        return tf.nn.relu(A) 
    
    # Helper function to get the network info
    def info(self):
        num_params = 0
        for i in range(1, self.L):
            num_params += self.W[i].shape[0] * self.W[i].shape[1]
            num_params += self.b[i].shape[0]
        print('Input Features:', self.num_features)
        print('Hidden Layers:')
        print('--------------')
        for i in range(1, self.L-1):
            print('Layer {}, Units {}'.format(i, self.layers[i]))
        print('--------------')
        print('Number of parameters:', num_params)
    
    # Train the network on batch of samples
    def train_on_batch(self, X, Y, lr):
        X = tf.convert_to_tensor(X, dtype=tf.float32)
        Y = tf.convert_to_tensor(Y, dtype=tf.float32)
        
        with tf.GradientTape(persistent=True) as tape:
            A = self.forward_pass(X)
            loss = self.compute_loss(A, Y)
        for i in range(1, self.L):
            self.dW[i] = tape.gradient(loss, self.W[i])
            self.db[i] = tape.gradient(loss, self.b[i])
        
        del tape
        self.update_params(lr)
        return loss.numpy()
    
    # Train the network on train and validate on test  samples
    def train(self, x_train, y_train, x_test, y_test, epochs, steps_per_epoch, batch_size, lr):
        history = {
            'val_loss': [],
            'train_loss': [],
            'val_acc': []
        }
        
        for e in range(0, epochs):
            epoch_train_loss = 0.
            print('Epoch {}'.format(e), end='.')
            for i in range(0, steps_per_epoch):
                x_batch = x_train[i*batch_size:(i+1)*batch_size]
                y_batch = y_train[i*batch_size:(i+1)*batch_size]
                
                batch_loss = self.train_on_batch(x_batch, y_batch, lr)
                epoch_train_loss += batch_loss
                
            history['train_loss'].append(epoch_train_loss/steps_per_epoch)
            val_A = self.forward_pass(x_test)
            val_loss = self.compute_loss(val_A, y_test).numpy()
            history['val_loss'].append(val_loss)
            val_preds = self.predict(x_test)
            #val_acc = np.mean(np.argmax(y_test, axis =1) == val_preds.numpy())
            val_acc = tf.compat.v1.losses.mean_squared_error(labels=y_test, predictions=val_preds.numpy()).numpy()
            history['val_acc'].append(val_acc)            
            train_preds = self.predict(x_train)
            train_acc = tf.compat.v1.losses.mean_squared_error(labels=y_train, predictions=train_preds.numpy()).numpy()
            print('Train acc: ', train_acc, '\tVal acc: ', val_acc)
        return history

In [ ]:
coln = 16

Train_X_inp = FPX_Train_X[:,:coln]
Train_X_spec = FPX_Train_X[:,coln:]
y_train = FPX_Train_y[:]
crude_train = FPX_Train_crudes[:]
Test_X_inp = FPX_Test_X[:,:coln]
Test_X_spec = FPX_Test_X[:,coln:]
y_test = FPX_Test_y[:]
crude_test = FPX_Test_crudes[:]

In [ ]:
Train_X_all = np.concatenate((np.array(Train_X_inp), np.array(Train_X_spec)), axis=1)
Test_X_all = np.concatenate((np.array(Test_X_inp), np.array(Test_X_spec)), axis=1)

In [ ]:
#(x_train, y_train), (x_test, y_test) = (Train_X_spec, y_train.reshape(-1,1)), (Test_X_spec, y_test.reshape(-1,1))
#(x_train, y_train), (x_test, y_test) = (Train_X_inp, y_train.reshape(-1,1)), (Test_X_inp, y_test.reshape(-1,1))
(x_train, y_train), (x_test, y_test) = (Train_X_all, y_train.reshape(-1,1)), (Test_X_all, y_test.reshape(-1,1))

In [ ]:
x_train = tf.constant(x_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
x_test = tf.constant(x_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
net = NeuralNetwork([int(x_train.shape[1]), 100, 50, 20, 5, 1])
net.info()

In [ ]:
batch_size =127
epochs = 100
steps_per_epoch = int(x_train.shape[0]//batch_size)
lr = 3e-3
print('Steps per epoch', steps_per_epoch)

In [ ]:
history = net.train(
    x_train, y_train, x_test, y_test,
    epochs, steps_per_epoch, batch_size, lr)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))

y_true_train, y_pred_train = y_train.numpy(), net.predict(x_train).numpy()
y_true_test, y_pred_test = y_test.numpy(), net.predict(x_test).numpy()

axs.scatter(y_true_train, y_pred_train, label = 'Train', s=20, facecolors='none', edgecolors='b')
axs.scatter(y_true_test, y_pred_test, label = 'Test', s=20, facecolors='none', edgecolors='r')

xlab = str('True ' + '' + '\n\n' + "MSE(train): " + 
    str(round(mean_squared_error(y_true_train, y_pred_train), 3))  
    + '\n $R^{2}(train):$' +
    str(round(get_r2(y_true_train, y_pred_train)[0], 3)) 
    + "\nMSE(test): " + 
    str(round(mean_squared_error(y_true_test, y_pred_test), 3))  
    + '\n $R^{2}(test):$' +
    str(round(get_r2(y_true_test, y_pred_test)[0], 3)))

xx1 = np.array([min(min(y_true_train),min(y_pred_train)) - 5,max(max(y_true_train),max(y_pred_train)) + 5])
xx2 = np.array([min(min(y_true_test),min(y_pred_test)) - 5,max(max(y_true_test),max(y_pred_test)) + 5])
xx  = np.array([min(xx1[0],xx2[0]), max(xx1[1],xx2[1])])
axs.plot(xx,xx,'--',color='grey')
axs.set_aspect('equal', 'box')
plt.xlim(xx[0], xx[1])
plt.ylim(xx[0], xx[1])
_ = axs.set(xlabel= xlab, ylabel='Prediction ' + '')

## Model Persistance

In [ ]:
W_x = net.W.copy()
b_x = net.b.copy()

for ky in W_x:
    W_x[ky] = W_x[ky].numpy().tolist()

for ky in b_x:
    b_x[ky] = b_x[ky].numpy().tolist()

model_dict = {'weights': W_x, 'bias': b_x}

In [ ]:
import json   
with open("model_dict.json", "w") as outfile:  
    json.dump(model_dict, outfile)

## Model Deployment

In [ ]:
import numpy as np
import pandas as pd
import json 

In [ ]:
def logit(z):
    return 1 / (1 + np.exp(-z))

def relu(weights): 
    return np.where(weights < 0., np.zeros_like(weights), weights)

In [ ]:
# Loading Weights and Bias
with open('model_dict.json') as f: 
    model_deploy = json.load(f) 

In [ ]:
class NeuralNetworkDeploy:
    # Initialization of network parameters
    def __init__(self, model_load):
        
        self.model_load = model_load
        
        layers = self.model_load['weights'].keys()
        self.L = len(layers) + 1
        
        self.W = {}
        self.b = {}
        
        self.setup()       
    # Load the weights 
    def setup(self):
        for i in range(1, self.L):
            self.W[i] = np.array(self.model_load['weights'][str(i)], dtype=np.float32)
            self.b[i] = np.array(self.model_load['bias'][str(i)], dtype=np.float32)
    
    # Forward Pass to iterate over each layer and get the final layer activation output
    def forward_pass(self, X):
        A = X
        for i in range(1, self.L):
            Z = np.matmul(A, np.transpose(self.W[i])) + np.transpose(self.b[i])
            if i != self.L-1:
                A = logit(Z)
            else:
                A = Z
        return A
    
    # Predict function: Forward pass --> ReLu
    def predict(self, X):
        A = self.forward_pass(X)
        return relu(A) 
    

In [ ]:
nn_dep = NeuralNetworkDeploy(model_deploy)

In [ ]:
nn_dep.predict(x_test)

In [ ]:
net.predict(x_test).numpy()